# SVM Browser

This notebook takes in a sample of normalized discrete wavelete transform approximation coefficients with corresponding data cleaning labels and outputs a trained SVM model. First, we load modules and files.

In [1]:
# Load modules
import time, pickle, json, lgdo
import numpy as np
from sklearn.svm import SVC


# Load files
dsp_file = '../data/l200-*-ml_train_dsp.lh5'
sto = lgdo.lh5.LH5Store()
tb_dsp, _ = sto.read('ml_train/dsp', dsp_file)

with open('../data/hyperparameters.json', 'r') as infile:
    hyperparams_dict = json.load(infile)

Here we define the training inputs that will go into the SVM.

In [2]:
dwts_norm = tb_dsp['dwt_norm'].nda
labels = tb_dsp['dc_label'].nda
SVM_hyperparams = hyperparams_dict['SVM']

Now we train the SVM with the optimal hyperparameters found, and then save the trained model.

In [3]:
svm = SVC(random_state=SVM_hyperparams['random_state'], 
          kernel=SVM_hyperparams['kernel'], 
          decision_function_shape=SVM_hyperparams['decision_function_shape'],
          class_weight=SVM_hyperparams['class_weight'],
          C=float(SVM_hyperparams['C']),
          gamma=float(SVM_hyperparams['gamma']))

start_time = time.time()
svm.fit(dwts_norm, labels)
print("--- %s seconds ---" % (time.time() - start_time))

with open("../data/svm.pkl", "wb") as svm_file:
    pickle.dump(svm, svm_file)

--- 3.4810590744018555 seconds ---


Now we save the trained model hyperparameters to be read by Julia as well.

In [4]:
svm_pars = svm.get_params()
svm_pars

{'C': 2969268.55357526,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': 'balanced',
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 0.06509150378608647,
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': 0,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [17]:
classes = svm.classes_
weights = svm.class_weight_

weights_dict = {}
for cl, wt in zip(classes, weights):
    weights_dict[cl] = wt
    
julia_svm_pars = {
    "degree" : svm_pars["degree"],
    "gamma" : svm_pars["gamma"],
    "cache_size" : svm_pars["cache_size"],
    "coef0" : svm_pars["coef0"],
    "cost" : svm_pars["C"],
    "tolerance" : svm_pars["tol"],
    "shrinking" : svm_pars["shrinking"],
    "probability" : svm_pars["probability"],
    "weights" : weights_dict,
    "verbose" : svm_pars["verbose"]
}
julia_svm_pars

Save the hyperparameters.

In [26]:
with open('../data/hyperparameters_julia.json', 'w') as f:
    json.dump(julia_svm_pars, f)

In [5]:
with open('../data/hyperparameters_pygama.json', 'w') as pf:
    json.dump(svm_pars, pf)